In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import os
from typing import List

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.cluster import KMeans
from multiprocessing import Pool as MultiprocessingPool, cpu_count

from src.kinematics import calculate_speed_and_direction

pd.set_option("display.max_columns", None)

/Users/kniu91/Documents/projects/bdb-26/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("./data")
    OUTPUT_DIR = Path("./outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 60
    PATIENCE = 30
    LEARNING_RATE = 1e-4
    
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    K_NEIGH = 6
    RADIUS = 30.0
    TAU = 8.0
    N_ROUTE_CLUSTERS = 7
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)



In [3]:
config = Config()
config

print("\n[1/4] Loading data...")
train_input_files = [config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
train_output_files = [config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])
supplementary_data = pd.read_csv(config.DATA_DIR / "supplementary_data.csv")

print(f"✓ Train input: {train_input.shape}, Train output: {train_output.shape}")
print(f"✓ Train output: {train_output.shape}, unique plays: {train_output[['game_id','play_id']].drop_duplicates().shape[0]}")
print(f"✓ Supplementary data: {supplementary_data.shape}")

traj_output = pd.read_csv('local_submission.csv')
traj_output = traj_output[['game_id', 'play_id', 'nfl_id', 'frame_id', 'pred_x', 'pred_y']]
traj_output.rename(columns={'pred_x': 'x', 'pred_y': 'y'}, inplace=True)

train_output.sort_values(by=['game_id', 'play_id', 'nfl_id', 'frame_id'], inplace=True)
traj_output.sort_values(by=['game_id', 'play_id', 'nfl_id', 'frame_id'], inplace=True)

print(f"✓ Projected trajectory output: {traj_output.shape}, unique plays: {traj_output[['game_id','play_id']].drop_duplicates().shape[0]}")


[1/4] Loading data...


/var/folders/_m/rvnpg_cs6xzcz0vlml0lzkbm0000gp/T/ipykernel_41314/1380030048.py:9: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplementary_data = pd.read_csv(config.DATA_DIR / "supplementary_data.csv")


✓ Train input: (4880579, 23), Train output: (562936, 6)
✓ Train output: (562936, 6), unique plays: 14108
✓ Supplementary data: (18009, 41)
✓ Projected trajectory output: (562936, 6), unique plays: 14108


In [4]:
def prepare_completions(suppl_df: pd.DataFrame) -> pd.DataFrame:
  play_results = suppl_df[['game_id','play_id','pass_result']].drop_duplicates()
  play_results.loc[play_results['pass_result'] == 'IN', 'pass_result'] = 'I'
  return play_results

def attach_and_prepare_play_level_features(input_df: pd.DataFrame,
                                           output_df:pd.DataFrame,
                                           supplementary_df: pd.DataFrame) -> pd.DataFrame:
    """
    Attaches play-level features from input_df and supplementary_df onto output_df

    Args:
        input_df (pd.DataFrame): Input DataFrame containing pre-throw tracking data
        output_df (pd.DataFrame): Output DataFrame containing post-throw tracking data
        supplementary_df (pd.DataFrame): Supplementary DataFrame containing supplementary play-level information
    """
    play_results = prepare_completions(supplementary_df)
    
    output_df = calculate_speed_and_direction(output_df)

    player_level_keys = ["game_id", "play_id", "nfl_id"]
    play_features = ["player_height", "player_weight","player_side","player_role",
                 "player_position",
                 "play_direction", "absolute_yardline_number",
                 "ball_land_x","ball_land_y"]

    input = input_df[player_level_keys + play_features].drop_duplicates()

    output_df = output_df.merge(input, on=player_level_keys, how='inner')
    output_df = output_df.merge(play_results, on=['game_id','play_id'], how='left', indicator= True)
    assert all(output_df['_merge'] == 'both')
    output_df = output_df.drop(columns=['_merge'])

    return output_df

after_pass_frames_real = attach_and_prepare_play_level_features(train_input, train_output, supplementary_data)
after_pass_frames_real['unique_play_id'] = after_pass_frames_real['game_id'].astype(str) + '_' + after_pass_frames_real['play_id'].astype(str)
train_output = after_pass_frames_real.copy()

after_pass_frames_proj = attach_and_prepare_play_level_features(train_input, traj_output, supplementary_data)
after_pass_frames_proj['unique_play_id'] = after_pass_frames_proj['game_id'].astype(str) + '_' + after_pass_frames_proj['play_id'].astype(str)
traj_output = after_pass_frames_proj

In [5]:
# ============================================================================
# GEOMETRIC BASELINE - THE BREAKTHROUGH
# ============================================================================

def compute_geometric_endpoint(df):
    """
    Compute where each player SHOULD end up based on geometry.
    This is the deterministic part - no learning needed.
    """
    df = df.copy()
    
    # Time to play end
    if 'num_frames_output' in df.columns:
        t_total = df['num_frames_output'] / 10.0
    else:
        t_total = 3.0
    
    df['time_to_endpoint'] = t_total
    
    # Initialize with momentum (default rule)
    df['geo_endpoint_x'] = df['x'] + df['velocity_x'] * t_total
    df['geo_endpoint_y'] = df['y'] + df['velocity_y'] * t_total
    
    # Rule 1: Targeted Receivers converge to ball
    if 'ball_land_x' in df.columns:
        receiver_mask = df['player_role'] == 'Targeted Receiver'
        df.loc[receiver_mask, 'geo_endpoint_x'] = df.loc[receiver_mask, 'ball_land_x']
        df.loc[receiver_mask, 'geo_endpoint_y'] = df.loc[receiver_mask, 'ball_land_y']
        
        # Rule 2: Defenders mirror receivers (maintain offset)
        defender_mask = df['player_role'] == 'Defensive Coverage'
        has_mirror = df.get('mirror_offset_x', 0).notna() & (df.get('mirror_wr_dist', 50) < 15)
        coverage_mask = defender_mask & has_mirror
        
        df.loc[coverage_mask, 'geo_endpoint_x'] = (
            df.loc[coverage_mask, 'ball_land_x'] + 
            df.loc[coverage_mask, 'mirror_offset_x'].fillna(0)
        )
        df.loc[coverage_mask, 'geo_endpoint_y'] = (
            df.loc[coverage_mask, 'ball_land_y'] + 
            df.loc[coverage_mask, 'mirror_offset_y'].fillna(0)
        )
    
    # Clip to field
    df['geo_endpoint_x'] = df['geo_endpoint_x'].clip(Config.FIELD_X_MIN, Config.FIELD_X_MAX)
    df['geo_endpoint_y'] = df['geo_endpoint_y'].clip(Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
    
    return df

def add_geometric_features(df):
    """Add features that describe the geometric solution"""
    df = compute_geometric_endpoint(df)
    
    # Vector to geometric endpoint
    df['geo_vector_x'] = df['geo_endpoint_x'] - df['x']
    df['geo_vector_y'] = df['geo_endpoint_y'] - df['y']
    df['geo_distance'] = np.sqrt(df['geo_vector_x']**2 + df['geo_vector_y']**2)
    
    # Required velocity to reach geometric endpoint
    t = df['time_to_endpoint'] + 0.1
    df['geo_required_vx'] = df['geo_vector_x'] / t
    df['geo_required_vy'] = df['geo_vector_y'] / t
    
    # Current velocity vs required
    df['geo_velocity_error_x'] = df['geo_required_vx'] - df['velocity_x']
    df['geo_velocity_error_y'] = df['geo_required_vy'] - df['velocity_y']
    df['geo_velocity_error'] = np.sqrt(
        df['geo_velocity_error_x']**2 + df['geo_velocity_error_y']**2
    )
    
    # Required constant acceleration (a = 2*Δx/t²)
    t_sq = t * t
    df['geo_required_ax'] = 2 * df['geo_vector_x'] / t_sq
    df['geo_required_ay'] = 2 * df['geo_vector_y'] / t_sq
    df['geo_required_ax'] = df['geo_required_ax'].clip(-10, 10)
    df['geo_required_ay'] = df['geo_required_ay'].clip(-10, 10)
    
    # Alignment with geometric path
    velocity_mag = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    geo_unit_x = df['geo_vector_x'] / (df['geo_distance'] + 0.1)
    geo_unit_y = df['geo_vector_y'] / (df['geo_distance'] + 0.1)
    df['geo_alignment'] = (
        df['velocity_x'] * geo_unit_x + df['velocity_y'] * geo_unit_y
    ) / (velocity_mag + 0.1)
    
    # Role-specific geometric quality
    df['geo_receiver_urgency'] = df['is_receiver'] * df['geo_distance'] / (t + 0.1)
    df['geo_defender_coupling'] = df['is_coverage'] * (1.0 / (df.get('mirror_wr_dist', 50) + 1.0))
    
    return df

In [6]:
def get_velocity(speed, direction_deg):
    theta = np.deg2rad(direction_deg)
    return speed * np.sin(theta), speed * np.cos(theta)

def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0

def get_opponent_features(input_df: pd.DataFrame) -> pd.DataFrame:
    """Enhanced opponent interaction with MIRROR WR tracking"""
    features = []
    
    for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']), 
                                   desc="🏈 Opponents", leave=False):
        last = group.sort_values('frame_id').groupby('nfl_id').last()
        
        if len(last) < 2:
            for nid in last.index:
                  features.append({
                      'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                      'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
                      'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
                      'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
                      'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
                      'mirror_wr_dist': 50.0,
                  })
            continue
            
        positions = last[['x', 'y']].values
        sides = last['player_side'].values
        speeds = last['s'].values
        directions = last['dir'].values
        roles = last['player_role'].values
        
        receiver_mask = np.isin(roles, ['Targeted Receiver', 'Other Route Runner'])
        
        for i, (nid, side, role) in enumerate(zip(last.index, sides, roles)):
            opp_mask = sides != side
            
            feat = {
                'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
                'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
                'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
                'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
                'mirror_wr_dist': 50.0,
            }
            
            if not opp_mask.any():
                features.append(feat)
                continue
            
            opp_positions = positions[opp_mask]
            distances = np.sqrt(((positions[i] - opp_positions)**2).sum(axis=1))
            
            if len(distances) == 0:
                features.append(feat)
                continue
                
            nearest_idx = distances.argmin()
            feat['nearest_opp_dist'] = distances[nearest_idx]
            feat['num_nearby_opp_3'] = (distances < 3.0).sum()
            feat['num_nearby_opp_5'] = (distances < 5.0).sum()
            
            my_vx, my_vy = get_velocity(speeds[i], directions[i])
            opp_speeds = speeds[opp_mask]
            opp_dirs = directions[opp_mask]
            opp_vx, opp_vy = get_velocity(opp_speeds[nearest_idx], opp_dirs[nearest_idx])
            
            rel_vx = my_vx - opp_vx
            rel_vy = my_vy - opp_vy
            to_me = positions[i] - opp_positions[nearest_idx]
            to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
            feat['closing_speed'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])
            
            if role == 'Defensive Coverage' and receiver_mask.any():
                rec_positions = positions[receiver_mask]
                rec_distances = np.sqrt(((positions[i] - rec_positions)**2).sum(axis=1))
                
                if len(rec_distances) > 0:
                    closest_rec_idx = rec_distances.argmin()
                    rec_indices = np.where(receiver_mask)[0]
                    actual_rec_idx = rec_indices[closest_rec_idx]
                    
                    rec_vx, rec_vy = get_velocity(speeds[actual_rec_idx], directions[actual_rec_idx])
                    
                    feat['mirror_wr_vx'] = rec_vx
                    feat['mirror_wr_vy'] = rec_vy
                    feat['mirror_wr_dist'] = rec_distances[closest_rec_idx]
                    feat['mirror_offset_x'] = positions[i][0] - rec_positions[closest_rec_idx][0]
                    feat['mirror_offset_y'] = positions[i][1] - rec_positions[closest_rec_idx][1]
            
            features.append(feat)
    
    return pd.DataFrame(features)

def compute_neighbor_embeddings(input_df, k_neigh=Config.K_NEIGH, 
                                radius=Config.RADIUS, tau=Config.TAU, print_logs=True):
    """GNN-lite embeddings"""
    if print_logs:
        print("🕸️  GNN embeddings...")
    
    cols_needed = ["game_id", "play_id", "nfl_id", "frame_id", "x", "y", 
                   "velocity_x", "velocity_y", "player_side"]
    src = input_df[cols_needed].copy()
    
    last = (src.sort_values(["game_id", "play_id", "nfl_id", "frame_id"])
               .groupby(["game_id", "play_id", "nfl_id"], as_index=False)
               .tail(1)
               .rename(columns={"frame_id": "last_frame_id"})
               .reset_index(drop=True))
    
    all_players = last[["game_id", "play_id", "nfl_id"]].copy()

    tmp = last.merge(
        src.rename(columns={
            "frame_id": "nb_frame_id", "nfl_id": "nfl_id_nb",
            "x": "x_nb", "y": "y_nb", 
            "velocity_x": "vx_nb", "velocity_y": "vy_nb", 
            "player_side": "player_side_nb"
        }),
        left_on=["game_id", "play_id", "last_frame_id"],
        right_on=["game_id", "play_id", "nb_frame_id"],
        how="left"
    )
    
    tmp = tmp[tmp["nfl_id_nb"] != tmp["nfl_id"]]
    tmp["dx"] = tmp["x_nb"] - tmp["x"]
    tmp["dy"] = tmp["y_nb"] - tmp["y"]
    tmp["dvx"] = tmp["vx_nb"] - tmp["velocity_x"]
    tmp["dvy"] = tmp["vy_nb"] - tmp["velocity_y"]
    tmp["dist"] = np.sqrt(tmp["dx"]**2 + tmp["dy"]**2)
    
    tmp = tmp[np.isfinite(tmp["dist"]) & (tmp["dist"] > 1e-6)]
    if radius is not None:
        tmp = tmp[tmp["dist"] <= radius]
    
    tmp["is_ally"] = (tmp["player_side_nb"] == tmp["player_side"]).astype(np.float32)
    
    keys = ["game_id", "play_id", "nfl_id"]
    tmp["rnk"] = tmp.groupby(keys)["dist"].rank(method="first")
    if k_neigh is not None:
        tmp = tmp[tmp["rnk"] <= float(k_neigh)]
    
    tmp["w"] = np.exp(-tmp["dist"] / float(tau))
    sum_w = tmp.groupby(keys)["w"].transform("sum")
    tmp["wn"] = np.where(sum_w > 0, tmp["w"] / sum_w, 0.0)
    
    tmp["wn_ally"] = tmp["wn"] * tmp["is_ally"]
    tmp["wn_opp"] = tmp["wn"] * (1.0 - tmp["is_ally"])
    
    for col in ["dx", "dy", "dvx", "dvy"]:
        tmp[f"{col}_ally_w"] = tmp[col] * tmp["wn_ally"]
        tmp[f"{col}_opp_w"] = tmp[col] * tmp["wn_opp"]
    
    tmp["dist_ally"] = np.where(tmp["is_ally"] > 0.5, tmp["dist"], np.nan)
    tmp["dist_opp"] = np.where(tmp["is_ally"] < 0.5, tmp["dist"], np.nan)
    
    ag = tmp.groupby(keys).agg(
        gnn_ally_dx_mean=("dx_ally_w", "sum"),
        gnn_ally_dy_mean=("dy_ally_w", "sum"),
        gnn_ally_dvx_mean=("dvx_ally_w", "sum"),
        gnn_ally_dvy_mean=("dvy_ally_w", "sum"),
        gnn_opp_dx_mean=("dx_opp_w", "sum"),
        gnn_opp_dy_mean=("dy_opp_w", "sum"),
        gnn_opp_dvx_mean=("dvx_opp_w", "sum"),
        gnn_opp_dvy_mean=("dvy_opp_w", "sum"),
        gnn_ally_cnt=("is_ally", "sum"),
        gnn_opp_cnt=("is_ally", lambda s: float(len(s) - s.sum())),
        gnn_ally_dmin=("dist_ally", "min"),
        gnn_ally_dmean=("dist_ally", "mean"),
        gnn_opp_dmin=("dist_opp", "min"),
        gnn_opp_dmean=("dist_opp", "mean"),
    ).reset_index()

    ag = all_players.merge(ag, on=keys, how="left")

    
    near = tmp.loc[tmp["rnk"] <= 3, keys + ["rnk", "dist"]].copy()
    if len(near) > 0:
        near["rnk"] = near["rnk"].astype(int)
        dwide = near.pivot_table(index=keys, columns="rnk", values="dist", aggfunc="first")
        dwide = dwide.rename(columns={1: "gnn_d1", 2: "gnn_d2", 3: "gnn_d3"}).reset_index()
        ag = ag.merge(dwide, on=keys, how="left")
    
    for c in ["gnn_ally_dx_mean", "gnn_ally_dy_mean", "gnn_ally_dvx_mean", "gnn_ally_dvy_mean",
              "gnn_opp_dx_mean", "gnn_opp_dy_mean", "gnn_opp_dvx_mean", "gnn_opp_dvy_mean"]:
        ag[c] = ag[c].fillna(0.0)
    for c in ["gnn_ally_cnt", "gnn_opp_cnt"]:
        ag[c] = ag[c].fillna(0.0)
    for c in ["gnn_ally_dmin", "gnn_opp_dmin", "gnn_ally_dmean", "gnn_opp_dmean", 
              "gnn_d1", "gnn_d2", "gnn_d3"]:
        if c in ag.columns:
            ag[c] = ag[c].fillna(radius if radius is not None else 30.0)
        else:
            # Create missing columns with default values
            ag[c] = radius if radius is not None else 30.0
  
    return ag



In [7]:
def prepare_sequences_geometric(input_df, 
                                output_df=None, 
                                test_template=None, 
                                is_training=True, 
                                window_size=5,
                                print_logs=True
                                # route_kmeans=None, 
                                # route_scaler=None
                                ):
    """
    YOUR 154 features + 13 geometric features = 167 total
    
    
    Returns:
        If Training:

        If Test:

    
    """
    if print_logs:
        print(f"\n{'='*80}")
        print(f"PREPARING GEOMETRIC SEQUENCES")
        print(f"{'='*80}")
    
    input_df = input_df.copy()
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    if print_logs:
        print("Step 1: Base features...")
    
    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    height_parts = input_df['player_height'].str.split('-', expand=True)
    input_df['height_inches'] = height_parts[0].astype(float) * 12 + height_parts[1].astype(float)
    input_df['bmi'] = (input_df['player_weight'] / (input_df['height_inches']**2)) * 703
    
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    input_df['velocity_x'] = input_df['s'] * np.sin(dir_rad)
    input_df['velocity_y'] = input_df['s'] * np.cos(dir_rad)
    # input_df['acceleration_x'] = input_df['a'] * np.cos(dir_rad)
    # input_df['acceleration_y'] = input_df['a'] * np.sin(dir_rad)
    
    input_df['speed_squared'] = input_df['s'] ** 2
    # input_df['accel_magnitude'] = np.sqrt(input_df['acceleration_x']**2 + input_df['acceleration_y']**2)
    input_df['momentum_x'] = input_df['velocity_x'] * input_df['player_weight']
    input_df['momentum_y'] = input_df['velocity_y'] * input_df['player_weight']
    input_df['kinetic_energy'] = 0.5 * input_df['player_weight'] * input_df['speed_squared']
    
    # input_df['orientation_diff'] = np.abs(input_df['o'] - input_df['dir'])
    # input_df['orientation_diff'] = np.minimum(input_df['orientation_diff'], 360 - input_df['orientation_diff'])
    
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    input_df['role_targeted_receiver'] = input_df['is_receiver']
    input_df['role_defensive_coverage'] = input_df['is_coverage']
    input_df['role_passer'] = input_df['is_passer']
    input_df['side_offense'] = input_df['is_offense']
    
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['dist_to_ball'] = input_df['distance_to_ball']
        input_df['dist_squared'] = input_df['distance_to_ball'] ** 2
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed_ball'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )
        input_df['velocity_toward_ball'] = (
            input_df['velocity_x'] * np.cos(input_df['angle_to_ball']) + 
            input_df['velocity_y'] * np.sin(input_df['angle_to_ball'])
        )
        input_df['velocity_alignment'] = np.cos(input_df['angle_to_ball'] - dir_rad)
        # input_df['angle_diff'] = np.abs(input_df['o'] - np.degrees(input_df['angle_to_ball']))
        # input_df['angle_diff'] = np.minimum(input_df['angle_diff'], 360 - input_df['angle_diff'])
    
    if print_logs:
        print("Step 2: Advanced features...")
    
    opp_features = get_opponent_features(input_df)
    input_df = input_df.merge(opp_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    # if is_training:
    #     route_features, route_kmeans, route_scaler = extract_route_patterns(input_df)
    # else:
    #     route_features = extract_route_patterns(input_df, route_kmeans, route_scaler, fit=False)
    # input_df = input_df.merge(route_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    gnn_features = compute_neighbor_embeddings(input_df, print_logs = print_logs)
    input_df = input_df.merge(gnn_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    if 'nearest_opp_dist' in input_df.columns:
        input_df['pressure'] = 1 / np.maximum(input_df['nearest_opp_dist'], 0.5)
        input_df['under_pressure'] = (input_df['nearest_opp_dist'] < 3).astype(int)
        input_df['pressure_x_speed'] = input_df['pressure'] * input_df['s']
    
    if 'mirror_wr_vx' in input_df.columns:
        s_safe = np.maximum(input_df['s'], 0.1)
        input_df['mirror_similarity'] = (
            input_df['velocity_x'] * input_df['mirror_wr_vx'] + 
            input_df['velocity_y'] * input_df['mirror_wr_vy']
        ) / s_safe
        input_df['mirror_offset_dist'] = np.sqrt(
            input_df['mirror_offset_x']**2 + input_df['mirror_offset_y']**2
        )
        input_df['mirror_alignment'] = input_df['mirror_similarity'] * input_df['role_defensive_coverage']
    
    if print_logs:
        print("Step 3: Temporal features...")
    
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # TODO: Add more lags/windows as needed?
    # for lag in [1, 2, 3, 4, 5]:
    #     for col in ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a']:
    #         if col in input_df.columns:
    #             input_df[f'{col}_lag{lag}'] = input_df.groupby(gcols)[col].shift(lag)
    
    # for window in [3, 5]:
    #     for col in ['x', 'y', 'velocity_x', 'velocity_y', 's']:
    #         if col in input_df.columns:
    #             input_df[f'{col}_rolling_mean_{window}'] = (
    #                 input_df.groupby(gcols)[col]
    #                   .rolling(window, min_periods=1).mean()
    #                   .reset_index(level=[0,1,2], drop=True)
    #             )
    #             input_df[f'{col}_rolling_std_{window}'] = (
    #                 input_df.groupby(gcols)[col]
    #                   .rolling(window, min_periods=1).std()
    #                   .reset_index(level=[0,1,2], drop=True)
    #             )
    
    for col in ['velocity_x', 'velocity_y']:
        if col in input_df.columns:
            input_df[f'{col}_delta'] = input_df.groupby(gcols)[col].diff()
    
    input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    if print_logs:
        print("Step 4: Time features...")
    
    if 'num_frames_output' in input_df.columns:
        max_frames = input_df['num_frames_output']
        
        input_df['max_play_duration'] = max_frames / 10.0
        input_df['frame_time'] = input_df['frame_id'] / 10.0
        input_df['progress_ratio'] = input_df['frame_id'] / np.maximum(max_frames, 1)
        input_df['time_remaining'] = (max_frames - input_df['frame_id']) / 10.0
        input_df['frames_remaining'] = max_frames - input_df['frame_id']
        
        input_df['expected_x_at_ball'] = input_df['x'] + input_df['velocity_x'] * input_df['frame_time']
        input_df['expected_y_at_ball'] = input_df['y'] + input_df['velocity_y'] * input_df['frame_time']
        
        if 'ball_land_x' in input_df.columns:
            input_df['error_from_ball_x'] = input_df['expected_x_at_ball'] - input_df['ball_land_x']
            input_df['error_from_ball_y'] = input_df['expected_y_at_ball'] - input_df['ball_land_y']
            input_df['error_from_ball'] = np.sqrt(
                input_df['error_from_ball_x']**2 + input_df['error_from_ball_y']**2
            )
            
            input_df['weighted_dist_by_time'] = input_df['dist_to_ball'] / (input_df['frame_time'] + 0.1)
            input_df['dist_scaled_by_progress'] = input_df['dist_to_ball'] * (1 - input_df['progress_ratio'])
        
        input_df['time_squared'] = input_df['frame_time'] ** 2
        input_df['velocity_x_progress'] = input_df['velocity_x'] * input_df['progress_ratio']
        input_df['velocity_y_progress'] = input_df['velocity_y'] * input_df['progress_ratio']
        input_df['speed_scaled_by_time_left'] = input_df['s'] * input_df['time_remaining']
        
        input_df['actual_play_length'] = max_frames
        input_df['length_ratio'] = max_frames / 30.0
    
    # 🎯 THE BREAKTHROUGH: Add geometric features
    if print_logs:
        print("Step 5: 🎯 Geometric endpoint features...")
    input_df = add_geometric_features(input_df)
    
    if print_logs:
        print("Step 6: Building feature list...")
    
    # Your 154 proven features
    feature_cols = [
        'x', 'y', 's', 
        # 'a', 'o', 
        'dir', 'frame_id', 'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight', 'height_inches', 'bmi',
        'velocity_x', 'velocity_y', 
        # 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'speed_squared', 'accel_magnitude', 
        # 'orientation_diff',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'role_targeted_receiver', 'role_defensive_coverage', 'role_passer', 'side_offense',
        'distance_to_ball', 'dist_to_ball', 'dist_squared', 'angle_to_ball', 
        'ball_direction_x', 'ball_direction_y', 'closing_speed_ball',
        'velocity_toward_ball', 'velocity_alignment', 
        # 'angle_diff',
        'nearest_opp_dist', 'closing_speed', 'num_nearby_opp_3', 'num_nearby_opp_5',
        'mirror_wr_vx', 'mirror_wr_vy', 'mirror_offset_x', 'mirror_offset_y',
        'pressure', 'under_pressure', 'pressure_x_speed', 
        'mirror_similarity', 'mirror_offset_dist', 'mirror_alignment',
        # 'route_pattern', 'traj_straightness', 'traj_max_turn', 'traj_mean_turn',
        # 'traj_depth', 'traj_width', 'speed_mean', 'speed_change',
        'gnn_ally_dx_mean', 'gnn_ally_dy_mean', 'gnn_ally_dvx_mean', 'gnn_ally_dvy_mean',
        'gnn_opp_dx_mean', 'gnn_opp_dy_mean', 'gnn_opp_dvx_mean', 'gnn_opp_dvy_mean',
        'gnn_ally_cnt', 'gnn_opp_cnt',
        'gnn_ally_dmin', 'gnn_ally_dmean', 'gnn_opp_dmin', 'gnn_opp_dmean',
        'gnn_d1', 'gnn_d2', 'gnn_d3',
    ]
    
    # for lag in [1, 2, 3, 4, 5]:
    #     for col in ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a']:
    #         feature_cols.append(f'{col}_lag{lag}')
    
    # for window in [3, 5]:
    #     for col in ['x', 'y', 'velocity_x', 'velocity_y', 's']:
    #         feature_cols.append(f'{col}_rolling_mean_{window}')
    #         feature_cols.append(f'{col}_rolling_std_{window}')
    
    feature_cols.extend(['velocity_x_delta', 'velocity_y_delta'])
    feature_cols.extend(['velocity_x_ema', 'velocity_y_ema', 'speed_ema'])
    
    feature_cols.extend([
        'max_play_duration', 'frame_time', 'progress_ratio', 'time_remaining', 'frames_remaining',
        'expected_x_at_ball', 'expected_y_at_ball', 
        'error_from_ball_x', 'error_from_ball_y', 'error_from_ball',
        'time_squared', 'weighted_dist_by_time', 
        'velocity_x_progress', 'velocity_y_progress', 'dist_scaled_by_progress',
        'speed_scaled_by_time_left', 'actual_play_length', 'length_ratio',
    ])
    
    # 🎯 Add 13 geometric features
    feature_cols.extend([
        'geo_endpoint_x', 'geo_endpoint_y',
        'geo_vector_x', 'geo_vector_y', 'geo_distance',
        'geo_required_vx', 'geo_required_vy',
        'geo_velocity_error_x', 'geo_velocity_error_y', 'geo_velocity_error',
        'geo_required_ax', 'geo_required_ay',
        'geo_alignment',
    ])
    
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    if print_logs:
        print(f"✓ Using {len(feature_cols)} features ({len(feature_cols) - 13} proven + 13 geometric)")
    
        print("Step 7: Creating sequences...")
    
    target_rows = input_df.copy() # Instantiate before we mess with input_df
    target_groups = target_rows[['game_id', 'play_id']].drop_duplicates()

    sequences, targets_catch, sequence_ids = [], [], []

    for _, row in tqdm(target_groups.iterrows(), total=len(target_groups), desc="Creating sequences", disable = not print_logs):
        # key = (row['game_id'], row['play_id'], row['nfl_id']) 
        key = (row['game_id'], row['play_id'])
        
        try:
            group_df = input_df[(input_df['game_id']==row['game_id']) &
                                 (input_df['play_id']==row['play_id'])]
        except KeyError:
            continue
        
        group_df = group_df[group_df['player_role']=='Targeted Receiver']
        input_window = group_df.tail(window_size)
        
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        
        if np.isnan(seq).any():
            if is_training:
                # Print the columns that have NaNs
                nan_cols = input_window[feature_cols].columns[input_window[feature_cols].isna().any()].tolist()
                print(f"Columns with NaNs: {nan_cols}")
                continue
            seq = np.nan_to_num(seq, nan=0.0)
        
        sequences.append(seq)
        
        # Store geometric endpoint for this player
        geo_x = input_window.iloc[-1]['geo_endpoint_x']
        geo_y = input_window.iloc[-1]['geo_endpoint_y']
        
        if is_training:
            out_grp = output_df[
                (output_df['game_id']==group_df.iloc[0]['game_id']) &
                (output_df['play_id']==group_df.iloc[0]['play_id']) &
                (output_df['nfl_id']==group_df.iloc[0]['nfl_id'])
            ].sort_values('frame_id')

            was_catch = out_grp['pass_result'].values[0] == 'C'
            targets_catch.append(1 if was_catch else 0)
            
        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'frame_id': input_window.iloc[-1]['frame_id']
        })

    if print_logs:
        print(f"✓ Created {len(sequences)} sequences")
    
    if is_training:
        return (sequences, 
                targets_catch,
                # targets_dx,
                # targets_dy, 
                # targets_frame_ids, 
                sequence_ids, 
                # geo_endpoints_x, 
                # geo_endpoints_y, 
                # route_kmeans, 
                # route_scaler,
                feature_cols)
    return sequences, sequence_ids#, geo_endpoints_x, geo_endpoints_y
    # return input_df

In [8]:
# ============================================================================
# MODEL ARCHITECTURE (YOUR PROVEN GRU + ATTENTION)
# ============================================================================
class JointSeqModel(nn.Module):
    """Your proven architecture - unchanged"""
    
    def __init__(self, input_dim: int):
        super().__init__()
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        
        self.head = nn.Sequential(
            nn.Linear(128, 256), 
            nn.GELU(), 
            nn.Dropout(0.2), 
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        ctx, _ = self.pool_attn(q, self.pool_ln(h), self.pool_ln(h))
        out = self.head(ctx.squeeze(1))
        return out.squeeze(-1)

In [9]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# ============================================================================
# TRAINING
# ============================================================================

def prepare_targets(batch_dx, batch_dy, max_h):
    tensors_x, tensors_y, masks = [], [], []
    
    for dx, dy in zip(batch_dx, batch_dy):
        L = len(dx)
        padded_x = np.pad(dx, (0, max_h - L), constant_values=0).astype(np.float32)
        padded_y = np.pad(dy, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        
        tensors_x.append(torch.tensor(padded_x))
        tensors_y.append(torch.tensor(padded_y))
        masks.append(torch.tensor(mask))
    
    targets = torch.stack([torch.stack(tensors_x), torch.stack(tensors_y)], dim=-1)
    return targets, torch.stack(masks)

def train_model(X_train: List[np.ndarray], 
                y_train: List[int], 
                X_val: List[np.ndarray], 
                y_val: List[int], 
                input_dim: int, 
                config: Config):
    device = config.DEVICE
    model = JointSeqModel(input_dim).to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by = torch.tensor(np.stack(y_train[i:end]).astype(np.float32))
        train_batches.append((bx, by))
    
    val_batches = []
    for i in range(0, len(X_val), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_val))
        bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
        by = torch.tensor(np.stack(y_val[i:end]).astype(np.float32))
        val_batches.append((bx, by))
    
    best_loss, best_state, bad = float('inf'), None, 0
    
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        for bx, by in train_batches:
            bx, by = bx.to(device), by.to(device)
            pred = model(bx)
            loss = criterion(pred, by)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for bx, by in val_batches:
                bx, by = bx.to(device), by.to(device)
                pred = model(bx)
                val_losses.append(criterion(pred, by).item())
                # ADD THESE 2 LINES:
                all_preds.append(torch.sigmoid(pred).cpu().numpy())
                all_targets.append(by.cpu().numpy())
              
        train_loss, val_loss = np.mean(train_losses), np.mean(val_losses)
        # ADD THESE LINES:
        y_pred_proba = np.concatenate(all_preds)
        y_true = np.concatenate(all_targets)
        y_pred = (y_pred_proba > 0.5).astype(int)
        
        auc = roc_auc_score(y_true, y_pred_proba)
        acc = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        
        # scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f"  Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f} | "
                f"AUC={auc:.3f}, Acc={acc:.3f}, Prec={precision:.3f}, Rec={recall:.3f}, F1={f1:.3f}")
      
        
        if val_loss < best_loss:
            best_loss = val_loss
            train_loss_at_best = train_loss
            auc_at_best = auc
            accuracy_at_best = acc
            precision_at_best = precision
            recall_at_best = recall
            f1_at_best = f1
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  Early stop at epoch {epoch}")
                break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model, best_loss, train_loss_at_best, auc_at_best, accuracy_at_best, precision_at_best, recall_at_best, f1_at_best, y_pred



In [10]:
from dataclasses import dataclass

sample_plays = False
number_sampmled = 10
if sample_plays:
    unique_plays = train_input[['game_id', 'play_id']].drop_duplicates()
    sampled_plays = unique_plays.sample(n=number_sampmled, random_state=42)
else:
    sampled_plays = train_input[['game_id', 'play_id']].drop_duplicates()

@dataclass
class TrajectoryObject:
    input_df: pd.DataFrame
    sequences: List[np.ndarray]
    targets_catch: List[int]
    sequence_ids: List[dict]
    feature_cols: List[str]

real_output_sampled = train_output.merge(sampled_plays, on=['game_id', 'play_id'], how='inner')
real_result = prepare_sequences_geometric(real_output_sampled, output_df = real_output_sampled)
realTrajectoryObject = TrajectoryObject(real_output_sampled, *real_result)

traj_output_sampled = traj_output.merge(sampled_plays, on=['game_id', 'play_id'], how='inner')
traj_result = prepare_sequences_geometric(traj_output_sampled, output_df = traj_output_sampled)
projTrajectoryObject = TrajectoryObject(
    traj_output_sampled, *traj_result
)



PREPARING GEOMETRIC SEQUENCES
Step 1: Base features...
Step 2: Advanced features...


🕸️  GNN embeddings...
Step 3: Temporal features...
Step 4: Time features...
Step 5: 🎯 Geometric endpoint features...
Step 6: Building feature list...
✓ Using 84 features (71 proven + 13 geometric)
Step 7: Creating sequences...


Creating sequences: 100%|██████████| 14108/14108 [01:40<00:00, 140.90it/s]


✓ Created 14108 sequences

PREPARING GEOMETRIC SEQUENCES
Step 1: Base features...
Step 2: Advanced features...


🕸️  GNN embeddings...
Step 3: Temporal features...
Step 4: Time features...
Step 5: 🎯 Geometric endpoint features...
Step 6: Building feature list...
✓ Using 84 features (71 proven + 13 geometric)
Step 7: Creating sequences...


Creating sequences: 100%|██████████| 14108/14108 [01:40<00:00, 140.54it/s]


✓ Created 14108 sequences


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def train_logistic_regression(X_train: List[np.ndarray], 
                              y_train: List[int], 
                              input_dim: int, 
                              config: Config,
                              cv_folds: int = 3):
    """
    Train logistic regression with cross-validation for hyperparameter tuning.
    
    Args:
        X_train: List of sequences (each shape: window_size x num_features)
        y_train: Binary labels (0/1)
        input_dim: Number of features (used for compatibility, not directly used)
        config: Config object
        cv_folds: Number of CV folds for GridSearchCV (default 3)
    
    Returns:
        model: Best fitted LogisticRegression model
        best_score: Best cross-validation AUC score
    """
    # Flatten sequences: (num_samples, window_size, features) -> (num_samples, window_size * features)
    X_train_flat = np.vstack([s.flatten() for s in X_train])
    y_train_array = np.array(y_train)
    
    print(f"  Flattened shape: {X_train_flat.shape}")
    
    # Define hyperparameter grid
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength
        'penalty': ['l2'],  # L2 regularization
        'max_iter': [2000],
        'solver': ['lbfgs']  # Good for L2
    }
    
    # Base model
    base_model = LogisticRegression(random_state=config.SEED, class_weight='balanced')
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(
        base_model,
        param_grid,
        cv=cv_folds,
        scoring='roc_auc',  # Optimize for AUC
        n_jobs=-1,
        verbose=1
    )
    
    print(f"  Running {cv_folds}-fold GridSearchCV...")
    grid_search.fit(X_train_flat, y_train_array)
    
    print(f"  ✓ Best params: {grid_search.best_params_}")
    print(f"  ✓ Best CV AUC: {grid_search.best_score_:.4f}")
    
    # Get best model
    best_model = grid_search.best_estimator_
    
    # Compute final training metrics
    y_train_pred_proba = best_model.predict_proba(X_train_flat)[:, 1]
    train_loss = log_loss(y_train_array, y_train_pred_proba)
    train_auc = roc_auc_score(y_train_array, y_train_pred_proba)
    
    print(f"  ✓ Final train loss: {train_loss:.4f}, AUC: {train_auc:.4f}")
    
    return best_model, grid_search.best_score_


In [12]:
def train_model_only(X_train: List[np.ndarray], 
                      y_train: List[int], 
                      input_dim: int, 
                      config: Config):
    device = config.DEVICE
    model = JointSeqModel(input_dim).to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, 
        T_max=config.EPOCHS,
        eta_min=1e-6  # Min LR at end
    )
    
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by = torch.tensor(np.stack(y_train[i:end]).astype(np.float32))
        train_batches.append((bx, by))
  
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        for bx, by in train_batches:
            bx, by = bx.to(device), by.to(device)
            pred = model(bx)
            loss = criterion(pred, by)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())              
        scheduler.step()
   
        train_loss = np.mean(train_losses)
     
        if epoch % 10 == 0:
            print(f"  Epoch {epoch}: train={train_loss:.4f}")
      
        return model, train_loss


In [13]:
from sklearn.metrics import log_loss

def predict_batch(model, X_data: List[np.ndarray], config: Config) -> np.ndarray:
    """
    Predict on a list of sequences with batching for efficiency.
    Works for any size - single play or thousands.
    """
    # model.eval()
    device = config.DEVICE
    all_preds = []
    
    with torch.no_grad():
        for i in range(0, len(X_data), config.BATCH_SIZE):
            end = min(i + config.BATCH_SIZE, len(X_data))
            batch = torch.tensor(np.stack(X_data[i:end]).astype(np.float32)).to(device)
            
            # Get logits and convert to probabilities
            logits = model(batch)
            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(probs)
    
    return np.concatenate(all_preds)

def predict_single(model, scaler, sequence: np.ndarray, config: Config) -> float:
    """
    Predict on a SINGLE sequence (for real-time inference).
    sequence shape: (window_size, num_features)
    """
    # model.eval()
    device = config.DEVICE
    
    # Scale and tensorize
    seq_scaled = scaler.transform(sequence)
    seq_tensor = torch.tensor(seq_scaled.astype(np.float32)).unsqueeze(0).to(device)  # Add batch dim
    
    with torch.no_grad():
        logit = model(seq_tensor)
        prob = torch.sigmoid(logit).cpu().item()  # Single scalar
    
    return prob

def compute_metrics(y_true: np.ndarray, y_pred_proba: np.ndarray, prefix: str = "") -> dict:
    """
    Compute classification metrics.
    
    Args:
        y_true: Ground truth labels (0 or 1)
        y_pred_proba: Predicted probabilities (0 to 1)
        prefix: Optional prefix for metric names (e.g., "val_", "real_")
    
    Returns:
        Dictionary of metrics
    """
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        f'{prefix}bce': log_loss(y_true, y_pred_proba),
        f'{prefix}auc': roc_auc_score(y_true, y_pred_proba),
        f'{prefix}acc': accuracy_score(y_true, y_pred),
    }
    
    return metrics

def print_metrics(metrics: dict, title: str = "Metrics"):
    """Pretty print metrics"""
    print(f"\n{title}:")
    for name, value in metrics.items():
        print(f"  {name}: {value:.4f}")

In [22]:
# def create_hybrid_trajectory(proj_df: pd.DataFrame, 
#                              real_df: pd.DataFrame, 
#                              game_id: int, 
#                              play_id: int, 
#                              swap_nfl_id: int) -> pd.DataFrame:
#     """
#     Create hybrid trajectory: all players projected EXCEPT swap_nfl_id (real).
    
#     Args:
#         proj_df: Projected trajectories for all players
#         real_df: Real trajectories for all players
#         game_id, play_id: Identify the play
#         swap_nfl_id: Which defender to use real trajectory for
    
#     Returns:
#         Hybrid DataFrame ready for feature engineering
#     """
#     # Start with projected
#     play_proj = proj_df[(proj_df['game_id'] == game_id) & 
#                         (proj_df['play_id'] == play_id)].copy()
    
#     # Get real trajectory for ONE defender
#     defender_real = real_df[(real_df['game_id'] == game_id) & 
#                            (real_df['play_id'] == play_id) & 
#                            (real_df['nfl_id'] == swap_nfl_id)].copy()
    
#     # Swap: remove projected version, add real version
#     play_hybrid = play_proj[play_proj['nfl_id'] != swap_nfl_id]
#     play_hybrid = pd.concat([play_hybrid, defender_real], ignore_index=True)
#     play_hybrid = play_hybrid.sort_values(['nfl_id', 'frame_id'])
    
#     return play_hybrid

def create_hybrid_trajectory(base_df: pd.DataFrame, 
                             swap_df: pd.DataFrame, 
                             game_id: int, 
                             play_id: int, 
                             swap_nfl_id: int) -> pd.DataFrame:
    """
    Create hybrid trajectory: all players projected EXCEPT swap_nfl_id (real).
    
    Args:
        base_df: The baseline trajectory df w. trajectories of all non-swapped players
        swap_df: The trajectory df containing swapped player's trajectory
        game_id, play_id: Identify the play
        swap_nfl_id: Which defender to use real trajectory for
    
    Returns:
        Hybrid DataFrame ready for feature engineering
    """
    # Start with projected
    play_proj = base_df[(base_df['game_id'] == game_id) & 
                        (base_df['play_id'] == play_id)].copy()
    
    # Get real trajectory for ONE defender
    defender_real = swap_df[(swap_df['game_id'] == game_id) & 
                           (swap_df['play_id'] == play_id) & 
                           (swap_df['nfl_id'] == swap_nfl_id)].copy()
    
    # Swap: remove projected version, add real version
    play_hybrid = play_proj[play_proj['nfl_id'] != swap_nfl_id]
    play_hybrid = pd.concat([play_hybrid, defender_real], ignore_index=True)
    play_hybrid = play_hybrid.sort_values(['nfl_id', 'frame_id'])
    
    return play_hybrid

def get_defender_impact(model: nn.Module, 
                        scaler: any, 
                        proj_df: pd.DataFrame, 
                        real_df: pd.DataFrame, 
                        game_id: int, 
                        play_id: int, 
                        config: any) -> pd.DataFrame:
    """
    Measure each defender's impact on catch probability.
    
    Returns:
        DataFrame with columns: [nfl_id, baseline_prob, real_prob, delta]
    """
    # Baseline: All projected
    play_proj = proj_df[(proj_df['game_id'] == game_id) & 
                        (proj_df['play_id'] == play_id)]
    
    # Get list of defenders
    defenders = play_proj['nfl_id'].unique()
    
    if len(defenders) == 0:
        return None
    
    # Baseline prediction (all projected)
    baseline_seq, _ = prepare_sequences_geometric(
        play_proj, output_df=play_proj, is_training=False, window_size=5, print_logs = False
    )
    baseline_seq_sc = [scaler.transform(s) for s in baseline_seq]
    baseline_flat = np.vstack([s.flatten() for s in baseline_seq_sc])
    # baseline_prob = predict_batch(model, baseline_seq_sc, config)[0]
    baseline_prob = model.predict_proba(baseline_flat)[:, 1][0]
    
    results = []
    
    for defender_id in defenders:
        # print(f"Processing for defender {defender_id}")
        # Create hybrid: this defender real, others projected
        hybrid_df = create_hybrid_trajectory(
            proj_df, real_df, game_id, play_id, defender_id
        )
        
        # Re-engineer features (GNN, opponents, etc.)
        hybrid_seq, _ = prepare_sequences_geometric(
            hybrid_df, output_df=hybrid_df, is_training=False, window_size=5, print_logs = False
        )
        
        # Predict
        hybrid_seq_sc = [scaler.transform(s) for s in hybrid_seq]
        hybrid_flat = np.vstack([s.flatten() for s in hybrid_seq_sc])
        # real_prob = predict_batch(model, hybrid_seq_sc, config)[0]
        real_prob = model.predict_proba(hybrid_flat)[:, 1][0]
        
        results.append({
            'nfl_id': defender_id,
            'player_role': play_proj[play_proj['nfl_id'] == defender_id]['player_role'].values[0],
            'baseline_prob': baseline_prob,
            'real_prob': real_prob,
            'delta': real_prob - baseline_prob,  # Negative = defender suppressed catch
        })
    
    return pd.DataFrame(results)

def get_defender_impact_flipped(model: nn.Module, 
                        scaler: any, 
                        proj_df: pd.DataFrame, 
                        real_df: pd.DataFrame, 
                        game_id: int, 
                        play_id: int, 
                        config: any) -> pd.DataFrame:
    """
    Measure each defender's impact on catch probability.

    In this case, the baseline is all REAL, and we swap in projected for each defender.
    
    Returns:
        DataFrame with columns: [nfl_id, baseline_prob, real_prob, delta]
    """
    # Baseline: All projected
    play_real = real_df[(real_df['game_id'] == game_id) & 
                        (real_df['play_id'] == play_id)]
    
    
    # Get list of defenders
    defenders = play_real['nfl_id'].unique()
    
    if len(defenders) == 0:
        return None
    
    # Baseline prediction (all projected)
    real_seq, _ = prepare_sequences_geometric(
        play_real, output_df=play_real, is_training=False, window_size=5, print_logs = False
    )
    real_seq_sc = [scaler.transform(s) for s in real_seq]
    real_flat = np.vstack([s.flatten() for s in real_seq_sc])
    real_prob = model.predict_proba(real_flat)[:, 1][0]
    
    results = []
    
    for defender_id in defenders:
        # print(f"Processing for defender {defender_id}")
        # Create hybrid: this defender real, others projected
        hybrid_df = create_hybrid_trajectory(
            real_df, proj_df, game_id, play_id, defender_id
        )
        
        # Re-engineer features (GNN, opponents, etc.)
        hybrid_seq, _ = prepare_sequences_geometric(
            hybrid_df, output_df=hybrid_df, is_training=False, window_size=5, print_logs = False
        )
        
        # Predict
        hybrid_seq_sc = [scaler.transform(s) for s in hybrid_seq]
        hybrid_flat = np.vstack([s.flatten() for s in hybrid_seq_sc])
        baseline_prob = model.predict_proba(hybrid_flat)[:, 1][0]
        # NOTE: Baseline here is the swapped projected version
        
        results.append({
            'nfl_id': defender_id,
            'player_role': play_real[play_real['nfl_id'] == defender_id]['player_role'].values[0],
            'baseline_prob': baseline_prob,
            'real_prob': real_prob,
            'delta': real_prob - baseline_prob,  # Negative = defender suppressed catch
        })
    
    return pd.DataFrame(results)

In [23]:
print("\n[3/4] Training geometric models...")
sequence_ids = realTrajectoryObject.sequence_ids
sequences = realTrajectoryObject.sequences

groups = np.array([d['game_id'] for d in sequence_ids])
gkf = GroupKFold(n_splits=config.N_FOLDS)

models, scalers = [], []

calibration_r2s = []  # ADD THIS BEFORE LOOP
importance_dfs = []  # STORE IMPORTANCE DFS

want_out_fold_metrics = True
for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{config.N_FOLDS}")
    print(f"{'='*60}")

    X_all = [sequences[i] for i in tr]; y_all = [realTrajectoryObject.targets_catch[i] for i in tr]
    X_out = [sequences[i] for i in va]; y_out = [realTrajectoryObject.targets_catch[i] for i in va]
    X_traj = [projTrajectoryObject.sequences[i] for i in va]

    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_all]))

    X_all_sc = [scaler.transform(s) for s in X_all]
    X_out_sc = [scaler.transform(s) for s in X_out]
    X_traj_sc = [scaler.transform(s) for s in X_traj]

    # model, train_loss = train_model_only(X_all_sc, y_all, X_all[0].shape[-1], config)
    model, cv_score = train_logistic_regression(
        X_all_sc, y_all, X_all[0].shape[-1], config, cv_folds=3
    )

    # Predict on real trajectories
    X_out_flat = np.vstack([s.flatten() for s in X_out_sc])
    preds_out = model.predict_proba(X_out_flat)[:, 1]
    # preds_out = predict_batch(model, X_out_sc, config)
    print(f"✓ Predicted on {len(preds_out)} validation plays")
    print(f"  Mean catch prob: {preds_out.mean():.3f}, Std: {preds_out.std():.3f}")
    if want_out_fold_metrics:
        # 🔥 EVALUATE: Real trajectories vs ground truth
        metrics_real = compute_metrics(np.array(y_out), preds_out, prefix="real_")
        print_metrics(metrics_real, title="📊 Validation Metrics (Real Trajectories)")

    # Predict on projected trajectories
    X_traj_flat = np.vstack([s.flatten() for s in X_traj_sc])
    preds_traj = model.predict_proba(X_traj_flat)[:, 1]
    # preds_traj = predict_batch(model, X_traj_sc, config)
    print(f"✓ Predicted on {len(preds_traj)} projected trajectories")

    # Join preds_out and preds_traj into a DataFrame for comparison, with game_id and play_id
    game_ids = [projTrajectoryObject.sequence_ids[i]['game_id'] for i in va]
    play_ids = [projTrajectoryObject.sequence_ids[i]['play_id'] for i in va]
    out_df = pd.DataFrame(zip(game_ids, play_ids, preds_traj, preds_out))
    out_df.columns = ['game_id', 'play_id', 'pred_catch_prob_by_proj_traj', 'pred_catch_prob_by_real_traj']
    # Write with appends to CSV
    out_df.to_csv(config.OUTPUT_DIR / f'catch_probabilities_log_flip.csv', index=False, header= fold == 1, mode='w' if fold == 1 else 'a')


    # ===================
    # The cool part - assessing defensive player impact
    # ===================
    val_game_ids = [projTrajectoryObject.sequence_ids[i]['game_id'] for i in va]
    val_play_ids = [projTrajectoryObject.sequence_ids[i]['play_id'] for i in va]
    
    all_impacts = []
    
    for num, (game_id, play_id) in enumerate(zip(val_game_ids, val_play_ids)):
        if num % 100 == 0:
            print(f"Processing defender impact for game {game_id}, play {play_id} (index {num})")
        impact_df = get_defender_impact_flipped(
            model, scaler, 
            projTrajectoryObject.input_df,  # Projected trajectories
            realTrajectoryObject.input_df,   # Real trajectories
            game_id, play_id, config
        )
        
        if impact_df is not None:
            impact_df['game_id'] = game_id
            impact_df['play_id'] = play_id
            impact_df['fold'] = fold
            impact_df = impact_df[['game_id', 'play_id', 'nfl_id', 'player_role', 'baseline_prob', 'real_prob', 'delta', 'fold']]
            all_impacts.append(impact_df)
    
    if all_impacts:
        fold_impact_df = pd.concat(all_impacts, ignore_index=True)
        # Save to CSV
        fold_impact_df.to_csv(config.OUTPUT_DIR / f'defender_impact_log_flip.csv', index=False, header = fold == 1, mode='w' if fold == 1 else 'a')
        # out_df.to_csv(config.OUTPUT_DIR / f'catch_probabilities.csv', index=False, header= fold == 1, mode='w' if fold == 1 else 'a')
        print(f"✓ Saved defender impact for fold {fold} with {len(fold_impact_df)} records")


    # preds_out = model.predict(X_out_sc, config)
    # preds_traj = model.predict(X_traj_sc, config)



[3/4] Training geometric models...

Fold 1/5
  Flattened shape: (11298, 420)
  Running 3-fold GridSearchCV...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
  ✓ Best params: {'C': 1.0, 'max_iter': 2000, 'penalty': 'l2', 'solver': 'lbfgs'}
  ✓ Best CV AUC: 0.8352
  ✓ Final train loss: 0.4754, AUC: 0.8456
✓ Predicted on 2810 validation plays
  Mean catch prob: 0.587, Std: 0.277

📊 Validation Metrics (Real Trajectories):
  real_bce: 0.4889
  real_auc: 0.8281
  real_acc: 0.7833
✓ Predicted on 2810 projected trajectories
Processing defender impact for game 2023091005, play 159 (index 0)


Processing defender impact for game 2023091006, play 4186 (index 100)


Processing defender impact for game 2023091705, play 2380 (index 200)


Processing defender impact for game 2023092401, play 2138 (index 300)


Processing defender impact for game 2023092411, play 401 (index 400)


...

Processing defender impact for game 2023122301, play 2590 (index 2500)


Processing defender impact for game 2023123101, play 2822 (index 2600)


Processing defender impact for game 2024010702, play 2748 (index 2700)


Processing defender impact for game 2024010712, play 273 (index 2800)


✓ Saved defender impact for fold 5 with 9363 records


In [149]:
c = pd.read_csv('./outputs/defender_impact_log_wr.csv')
# c.columns = ['game_id', 'play_id', 'nfl_id', 'baseline_prob', 'real_prob', 'delta', 'fold']
# c.head()
c = c.merge(supplementary_data[['game_id','play_id','pass_result', 'yards_gained', 'season','week','home_team_abbr','visitor_team_abbr','play_description', 'quarter','game_clock']],
        on=['game_id','play_id'], how='left')
# c[(c['game_id'] == 2023091711) & (c['play_id'] == 1082)]
c.sort_values('delta', ascending = False).head(20)

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock
28066,2023091701,3025,41282,Targeted Receiver,0.061843,0.844657,0.782814,4,I,0,2023,2,BUF,LV,(12:57) (Shotgun) J.Garoppolo pass incomplete ...,4,12:57
38324,2023091711,1082,55928,Targeted Receiver,0.123415,0.790457,0.667041,5,C,53,2023,2,DEN,WAS,(13:15) (Shotgun) R.Wilson pass deep left to M...,2,13:15
30835,2023101600,4288,47911,Targeted Receiver,0.179383,0.796526,0.617143,4,I,0,2023,6,LAC,DAL,(2:28) (Shotgun) D.Prescott pass incomplete de...,4,02:28
11334,2023100805,2735,55133,Targeted Receiver,0.093919,0.688525,0.594607,2,C,3,2023,5,NE,NO,(9:12) (Shotgun) D.Carr pass short right to R....,3,09:12
29581,2023100110,2087,48097,Targeted Receiver,0.081735,0.666244,0.584508,4,C,8,2023,4,LAC,LV,(1:11) (Shotgun) A.O'Connell pass short right ...,2,01:11
39829,2023100111,205,52425,Targeted Receiver,0.249542,0.821715,0.572173,5,I,0,2023,4,DAL,NE,"(12:16) (No Huddle, Shotgun) D.Prescott pass i...",1,12:16
20257,2023101508,2360,41233,Targeted Receiver,0.224503,0.788287,0.563785,3,I,0,2023,6,TB,DET,(7:52) (Shotgun) B.Mayfield pass incomplete sh...,3,07:52
30629,2023101505,3419,39989,Targeted Receiver,0.048402,0.609544,0.561142,4,I,0,2023,6,HOU,NO,(9:40) (Shotgun) C.Stroud pass incomplete deep...,4,09:40
783,2023091707,1259,53506,Targeted Receiver,0.326438,0.881550,0.555111,1,I,0,2023,2,TEN,LAC,(8:29) (Shotgun) J.Herbert pass incomplete sho...,2,08:29
3198,2023102201,3284,41282,Targeted Receiver,0.309605,0.844710,0.535104,1,I,0,2023,7,CHI,LV,(12:28) (Shotgun) B.Hoyer pass incomplete shor...,4,12:28


In [ ]:
# # join all impacts
# if all_impacts:
#     fold_impact_df = pd.concat(all_impacts, ignore_index=True)
#     importance_dfs.append(fold_impact_df)

pd.concat(all_impacts)

In [ ]:
def compute_feature_importance(model, X_val, y_val, feature_names, device, n_repeats=3):
    """Permutation-based feature importance"""
    from sklearn.metrics import roc_auc_score
    
    # model.eval()
    X_val_tensor = torch.tensor(np.stack(X_val).astype(np.float32)).to(device)
    y_val_array = np.array(y_val)
    
    # Baseline score
    with torch.no_grad():
        baseline_pred = torch.sigmoid(model(X_val_tensor)).cpu().numpy()
        baseline_score = roc_auc_score(y_val_array, baseline_pred)
    
    importances = []
    
    for feat_idx in tqdm(range(len(feature_names)), desc="Computing importances"):
        scores = []
        for _ in range(n_repeats):
            X_permuted = [x.copy() for x in X_val]
            # Permute this feature across all sequences
            perm_values = np.random.permutation([x[:, feat_idx] for x in X_val])
            for i, x in enumerate(X_permuted):
                x[:, feat_idx] = perm_values[i]
            
            X_perm_tensor = torch.tensor(np.stack(X_permuted).astype(np.float32)).to(device)
            with torch.no_grad():
                perm_pred = torch.sigmoid(model(X_perm_tensor)).cpu().numpy()
                perm_score = roc_auc_score(y_val_array, perm_pred)
            scores.append(baseline_score - perm_score)  # Drop in performance
        
        importances.append(np.mean(scores))
    
    # Create DataFrame
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    return importance_df


In [ ]:
print("\n[3/4] Training geometric models...")
groups = np.array([d['game_id'] for d in sequence_ids])
gkf = GroupKFold(n_splits=config.N_FOLDS)

models, scalers = [], []

calibration_r2s = []  # ADD THIS BEFORE LOOP
importance_dfs = []  # STORE IMPORTANCE DFS

for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{config.N_FOLDS}")
    print(f"{'='*60}")

    X_tr = [sequences[i] for i in tr]
    X_va = [sequences[i] for i in va]
    y_tr = [targets_catch[i] for i in tr]
    y_va = [targets_catch[i] for i in va]

    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_tr]))

    X_tr_sc = [scaler.transform(s) for s in X_tr]
    X_va_sc = [scaler.transform(s) for s in X_va]
        
    model, loss, train_loss, auc, acc, prec, rec, f1, y_pred_proba = train_model(
        X_tr_sc, y_tr,
        X_va_sc, y_va,
        X_tr[0].shape[-1], config = config
    )
    
    print(f"\n✓ Fold {fold} validation outcomes - BCE Loss: {loss:.5f}, Train Loss: {train_loss:.5f}, AUC: {auc:.3f}, Acc: {acc:.3f}, Prec: {prec:.3f}, Rec: {rec:.3f}, F1: {f1:.3f}")
    
    # ==================================
    # 🔥 CALIBRATION CHECK
    # ==================================
    model.eval()
    # X_va_tensor = torch.tensor(np.stack(X_va_sc).astype(np.float32)).to(config.DEVICE)
    # with torch.no_grad():
    #     y_va_pred_proba = torch.sigmoid(model(X_va_tensor)).cpu().numpy()
    
    # bins = np.linspace(0, 1, 11)
    # bin_indices = np.digitize(y_va_pred_proba, bins) - 1
    
    # predicted_probs, actual_rates = [], []
    # for i in range(10):
    #     mask = (bin_indices == i)
    #     if mask.sum() > 0:
    #         predicted_probs.append(y_va_pred_proba[mask].mean())
    #         actual_rates.append(np.array(y_va)[mask].mean())
    
    # from sklearn.metrics import r2_score
    # if len(predicted_probs) > 1:
    #     r2 = r2_score(actual_rates, predicted_probs)
    #     calibration_r2s.append(r2)  # 🔥 STORE IT
    #     print(f"📈 Calibration R² = {r2:.3f}")
    # ==================================
    # 🔥 CALIBRATION CHECK
    # ==================================
    
    models.append(model)
    scalers.append(scaler)
     
    # Feature importance code...
    print(f"\n📊 Computing feature importance for fold {fold}...")
    importance_df = compute_feature_importance(
        model, X_va_sc, y_va, feature_cols, config.DEVICE, n_repeats=3
    )
    importance_dfs.append(importance_df)
    
# 🔥 FINAL SUMMARY
print("\n" + "="*80)
print("FINAL MODEL SUMMARY")
print("="*80)
print(f"Mean Calibration R²: {np.mean(calibration_r2s):.3f} ± {np.std(calibration_r2s):.3f}")
print(f"🏆 NFL Benchmark: 0.98 | Your Gap: {0.98 - np.mean(calibration_r2s):.3f}")

all_importances = []
for fold in range(1, config.N_FOLDS + 1):
    df = importance_dfs[fold-1]
    all_importances.append(df)

avg_importance = pd.concat(all_importances).groupby('feature')['importance'].mean()
avg_importance = avg_importance.sort_values(ascending=False).reset_index()

print(avg_importance.head(30))
avg_importance.to_csv(config.OUTPUT_DIR / 'feature_importance_avg.csv', index=False)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss


for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{config.N_FOLDS}")
    print(f"{'='*60}")

    X_tr = [sequences[i] for i in tr]
    X_va = [sequences[i] for i in va]
    y_tr = [targets_catch[i] for i in tr]
    y_va = [targets_catch[i] for i in va]

    # Flatten sequences: (num_samples, 10 timesteps, 153 features) -> (num_samples, 1530)
    X_tr_flat = np.vstack([s.flatten() for s in X_tr])
    X_va_flat = np.vstack([s.flatten() for s in X_va])
    
    scaler = StandardScaler()
    X_tr_sc = scaler.fit_transform(X_tr_flat)
    X_va_sc = scaler.transform(X_va_flat)
    
    # Grid search for best C (inverse regularization strength)
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # L2 penalty strength
        'max_iter': [1000]
    }
    
    base_model = LogisticRegression(penalty='l2', solver='lbfgs', random_state=config.SEED)
    
    grid_search = GridSearchCV(
        base_model, 
        param_grid, 
        cv=3,  # 3-fold CV within training data
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )
    
    print(f"  Running GridSearchCV...")
    grid_search.fit(X_tr_sc, y_tr)
    
    print(f"  Best params: {grid_search.best_params_}")
    print(f"  Best CV score: {grid_search.best_score_:.4f}")
    
    # Use best model
    model = grid_search.best_estimator_
    
    # Evaluate on validation set
    y_va_pred_proba = model.predict_proba(X_va_sc)[:, 1]
    y_va_pred = (y_va_pred_proba > 0.5).astype(int)
    
    # 🔥 ADD BCE COMPUTATION
    bce_loss = log_loss(y_va, y_va_pred_proba)
    
    val_auc = roc_auc_score(y_va, y_va_pred_proba)
    acc = accuracy_score(y_va, y_va_pred)
    precision = precision_score(y_va, y_va_pred, zero_division=0)
    recall = recall_score(y_va, y_va_pred, zero_division=0)
    f1 = f1_score(y_va, y_va_pred, zero_division=0)
    
    # 🔥 UPDATE PRINT TO INCLUDE BCE
    print(f"  Validation: BCE={bce_loss:.4f}, AUC={val_auc:.4f}, Acc={acc:.3f}, "
          f"Prec={precision:.3f}, Rec={recall:.3f}, F1={f1:.3f}")
    
    models.append(model)
    scalers.append(scaler)
    
    # Feature importance from coefficients
    coef = model.coef_[0]  # Shape: (1530,) for flattened features
    
    # Aggregate across timesteps: (10 timesteps, 153 features) -> (153,)
    coef_reshaped = coef.reshape(10, len(feature_cols))
    feature_importance = np.abs(coef_reshaped).mean(axis=0)  # Average across time
    
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print("\n🏆 Top 20 Features:")
    print(importance_df.head(20).to_string(index=False))
    
    importance_df.to_csv(config.OUTPUT_DIR / f'importance_fold{fold}_logreg.csv', index=False)

# After all folds, aggregate
print("\n" + "="*80)
print("AGGREGATED FEATURE IMPORTANCE ACROSS FOLDS")
print("="*80)

all_importances = []
for fold in range(1, config.N_FOLDS + 1):
    df = pd.read_csv(config.OUTPUT_DIR / f'importance_fold{fold}_logreg.csv')
    all_importances.append(df)

avg_importance = pd.concat(all_importances).groupby('feature')['importance'].mean()
avg_importance = avg_importance.sort_values(ascending=False).reset_index()
print(avg_importance.head(30))


In [ ]:
HORIZON, config
    )
    
    models.append(model)
    scalers.append(scaler)
    
    print(f"\n✓ Fold {fold} - Loss: {loss:.5f}")

In [ ]:

for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{config.N_FOLDS}")
    print(f"{'='*60}")a
    
    X_tr = [sequences[i] for i in tr]
    X_va = [sequences[i] for i in va]
    y_tr_dx = [targets_dx[i] for i in tr]
    y_va_dx = [targets_dx[i] for i in va]
    y_tr_dy = [targets_dy[i] for i in tr]
    y_va_dy = [targets_dy[i] for i in va]
    
    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_tr]))
    
    X_tr_sc = [scaler.transform(s) for s in X_tr]
    X_va_sc = [scaler.transform(s) for s in X_va]
    
    model, loss = train_model(
        X_tr_sc, y_tr_dx, y_tr_dy,
        X_va_sc, y_va_dx, y_va_dy,
        X_tr[0].shape[-1], config.MAX_FUTURE_HORIZON, config
    )
    
    models.append(model)
    scalers.append(scaler)
    
    print(f"\n✓ Fold {fold} - Loss: {loss:.5f}")

In [ ]:
a = prepare_sequences_geometric(train_input, train_output, is_training=True, window_size=10)

In [ ]:
a.head()

In [ ]:
# ==========================================
# Get Opponent Features
# ==========================================
# This includes mirror / wr tracking
# Seems like wecan include because it only has speed and positions?

# TODO: Implement opponent features
# Hopefully we just need to update some variable names after getting kinematics in


In [ ]:
# ==========================================
# Extract route patterns
# ==========================================



In [ ]:
# ==========================================
# Compute neighbor embeddings
# ==========================================
# This is the GNN

In [ ]:
# ==========================================
# Create temporal features (rolling mean)
# ==========================================

In [ ]:
# ==========================================
# Time features
# ==========================================

In [ ]:
# ==========================================
# Geometric features
# ==========================================